In [ ]:
# Exercise 1
# In a senario where there are more variables than observations, using the least squares method would not 
# be ideal because of the tendancy to have a low bias, but high variance and not having a unique solution.
# In this case, the best method to use is ridge regression because however it will have a high bias, the 
# variance will be much lower than least squares.

In [ ]:
# Exercise 2
# (f)